In [1]:
import sys
print(sys.executable)

/Users/chunghyunhan/Projects/agentics/venv/bin/python


In [5]:
import requests
import pandas as pd

# Snapshot GraphQL API endpoint
SNAPSHOT_API = "https://hub.snapshot.org/graphql"

# GraphQL query
query = """
{
  spaces(first: 30, orderBy: "votesCount", orderDirection: desc) {
    id
    name
    votesCount
    proposalsCount
    followersCount
  }
}
"""

# API 호출
response = requests.post(SNAPSHOT_API, json={"query": query})

# 결과 JSON 파싱
data = response.json()

# space 정보 추출
spaces = data["data"]["spaces"]

# DataFrame으로 정리
df_daos = pd.DataFrame(spaces)
print(df_daos)

# 상위 30 DAO 리스트
top_30_daos = df_daos["id"].tolist()
print("Top 30 DAOs:", top_30_daos)


                       id                         name  votesCount  \
0     labs.opencivics.eth              OpenCivics Labs           0   
1         grigorykair.eth                         Test           0   
2              soneta.eth                       Soneta           0   
3         obeliskvote.eth                      Obelisk           0   
4                womo.eth                         WOMO          57   
5        moneyflowdao.eth              MoneyFLOW - DAO           0   
6             votebox.eth                      Votebox           0   
7            diamonde.eth                 diamonde DAO           0   
8         aventus-gov.eth              Aventus Network           0   
9          chronaraai.eth              Chronara AI DAO           0   
10           froglabs.eth               Ethereum Frogs           0   
11        critswallet.eth                         MTGG           0   
12        basedchadhq.eth                    Chad Test           0   
13              zugg

In [ ]:
# Save data Locally 
import requests
import os
import json
from typing import List, Dict
from dataclasses import dataclass, asdict
import pandas as pd

# --- 설정 ---

SNAPSHOT_API = "https://hub.snapshot.org/graphql"

# 조회할 spaces
SPACES = [
    "aavedao.eth",
    "arbitrumfoundation.eth",
    "snapshot.dcl.eth",
    "balancer.eth",
    "cvx.eth",
    "1inch.eth",
    "aurafinance.eth",
    "lido-snapshot.eth",
    "uniswapgovernance.eth",
    "metislayer2.eth"
]

# 결과 저장 폴더
OUT_DIR = "snapshot_data"
os.makedirs(OUT_DIR, exist_ok=True)

# --- 데이터 모델 ---

@dataclass
class Vote:
    voter: str
    vp: float
    choice: str    # 수정: int → str
    reason: str
    created: int

@dataclass
class ProposalData:
    id: str
    space: str
    title: str
    body: str
    choices: List[str]
    author: str
    start: int
    end: int
    created: int
    state: str
    votes: List[Vote]

# --- 함수들 ---

def graphql_query(query: str, variables: Dict = None) -> Dict:
    response = requests.post(SNAPSHOT_API, json={"query": query, "variables": variables or {}})
    response.raise_for_status()
    return response.json()

def get_proposals_for_space(space: str, first: int = 50) -> List[Dict]:
    query = """
    query ($space: String!, $first: Int!) {
      proposals(first: $first, where: {space_in: [$space]}, orderBy: "created", orderDirection: desc) {
        id
        title
        body
        choices
        author
        start
        end
        created
        state
      }
    }
    """
    result = graphql_query(query, {"space": space, "first": first})
    return result["data"]["proposals"]

def get_votes_for_proposal(proposal_id: str, first: int = 1000) -> List[Dict]:
    query = """
    query ($id: String!, $first: Int!) {
      votes(first: $first, where: {proposal: $id}, orderBy: "created", orderDirection: asc) {
        voter
        vp
        choice
        reason
        created
      }
    }
    """
    result = graphql_query(query, {"id": proposal_id, "first": first})
    return result["data"]["votes"]

# --- 실행 및 저장 ---

def fetch_and_save_all():
    all_proposals: List[ProposalData] = []
    for space in SPACES:
        print(f"Fetching proposals for space: {space}")
        proposals = get_proposals_for_space(space, first=50)
        for prop in proposals:
            print(f"  Proposal {prop['id']} ... fetching votes")
            votes_raw = get_votes_for_proposal(prop["id"], first=1000)
            votes = []
            for vr in votes_raw:
                choice_val = vr.get("choice", 0)
                # choice가 list인 경우 → 문자열로 변환
                if isinstance(choice_val, list):
                    choice_val = ",".join(map(str, choice_val))
                else:
                    choice_val = str(choice_val)

                vote = Vote(
                    voter=vr.get("voter", ""),
                    vp=float(vr.get("vp", 0) or 0),
                    choice=choice_val,
                    reason=vr.get("reason", "") or "",
                    created=int(vr.get("created", 0) or 0),
                )
                votes.append(vote)

            prop_data = ProposalData(
                id=prop["id"],
                space=space,
                title=prop.get("title", ""),
                body=prop.get("body", ""),
                choices=prop.get("choices", []),
                author=prop.get("author", ""),
                start=int(prop.get("start", 0) or 0),
                end=int(prop.get("end", 0) or 0),
                created=int(prop.get("created", 0) or 0),
                state=prop.get("state", ""),
                votes=votes
            )
            all_proposals.append(prop_data)

        # space별 저장 (proposals)
        p_df = pd.DataFrame([{
            "id": p.id,
            "space": p.space,
            "title": p.title,
            "body": p.body,
            "choices": json.dumps(p.choices),
            "author": p.author,
            "start": p.start,
            "end": p.end,
            "created": p.created,
            "state": p.state
        } for p in all_proposals if p.space == space])
        p_df.to_csv(os.path.join(OUT_DIR, f"proposals_{space}.csv"), index=False)
        
        # votes 저장
        v_rows = []
        for p in all_proposals:
            if p.space != space:
                continue
            for v in p.votes:
                v_rows.append({
                    "proposal_id": p.id,
                    "voter": v.voter,
                    "vp": v.vp,
                    "choice": v.choice,
                    "reason": v.reason,
                    "created": v.created
                })
        v_df = pd.DataFrame(v_rows)
        v_df.to_csv(os.path.join(OUT_DIR, f"votes_{space}.csv"), index=False)

    # 전체 저장 (JSON + Parquet)
    with open(os.path.join(OUT_DIR, "all_proposals_with_votes.json"), "w") as f:
        json.dump([asdict(p) for p in all_proposals], f, default=str)

    all_prop_df = pd.DataFrame([{
        "id": p.id,
        "space": p.space,
        "title": p.title,
        "body": p.body,
        "choices": json.dumps(p.choices),
        "author": p.author,
        "start": p.start,
        "end": p.end,
        "created": p.created,
        "state": p.state
    } for p in all_proposals])
    all_prop_df.to_parquet(os.path.join(OUT_DIR, "all_proposals.parquet"), index=False)

    print("✅ Done saving data.")

# --- 메인 실행 ---
if __name__ == "__main__":
    fetch_and_save_all()


In [ ]:
import requests
from datetime import datetime, timezone
from typing import Dict, List

SNAPSHOT_API = "https://hub.snapshot.org/graphql"

SPACES = [
    "aavedao.eth",
    "arbitrumfoundation.eth",
    "snapshot.dcl.eth",
    "balancer.eth",
    "cvx.eth",
    "1inch.eth",
    "aurafinance.eth",
    "lido-snapshot.eth",
    "uniswapgovernance.eth",
    "metislayer2.eth",
]

# --- helpers --------------------------------------------------------------

def ts_to_iso(ts: int) -> str:
    return datetime.fromtimestamp(int(ts), tz=timezone.utc).isoformat()

def gql(query: str, variables=None) -> dict:
    r = requests.post(SNAPSHOT_API, json={"query": query, "variables": variables or {}})
    r.raise_for_status()
    data = r.json()
    if "errors" in data:
        raise RuntimeError(data["errors"])
    return data["data"]

# Snapshot supports pagination with (first, skip)
PROPOSALS_Q = """
query($space: String!, $first: Int!, $skip: Int!) {
  proposals(
    first: $first
    skip: $skip
    where: { space_in: [$space] }
    orderBy: "created"
    orderDirection: desc
  ) {
    id
    title
    state
    start
    end
    created
  }
}
"""

def fetch_all_proposals(space: str, batch: int = 100) -> List[dict]:
    results, skip = [], 0
    while True:
        d = gql(PROPOSALS_Q, {"space": space, "first": batch, "skip": skip})
        chunk = d["proposals"]
        if not chunk:
            break
        results.extend(chunk)
        if len(chunk) < batch:
            break
        skip += batch
    return results

def finished_only(proposals: List[dict]) -> List[dict]:
    # Snapshot marks completed proposals as "closed".
    # We also guard by end <= now (in case of state inconsistencies).
    now_ts = int(datetime.now(timezone.utc).timestamp())
    return [
        p for p in proposals
        if (p.get("state") == "closed") and int(p.get("end", 0) or 0) <= now_ts
    ]

def summarize_space(space: str) -> Dict:
    all_props = fetch_all_proposals(space)
    fin = finished_only(all_props)
    if not fin:
        return {
            "space": space,
            "finished_count": 0,
            "earliest": None,
            "latest": None,
        }

    # Sort by end time to define earliest/latest among finished
    fin_sorted = sorted(fin, key=lambda p: int(p["end"]))

    earliest = fin_sorted[0]
    latest = fin_sorted[-1]

    return {
        "space": space,
        "finished_count": len(fin),
        "earliest": {
            "id": earliest["id"],
            "title": earliest["title"],
            "end_ts": earliest["end"],
            "end_iso": ts_to_iso(earliest["end"]),
        },
        "latest": {
            "id": latest["id"],
            "title": latest["title"],
            "end_ts": latest["end"],
            "end_iso": ts_to_iso(latest["end"]),
        },
    }

# --- run ------------------------------------------------------------------

if __name__ == "__main__":
    rows = [summarize_space(s) for s in SPACES]
    # Pretty print
    for r in rows:
        print(f"\n[{r['space']}] finished={r['finished_count']}")
        if r["earliest"]:
            e = r["earliest"]
            l = r["latest"]
            print(f"  earliest: {e['end_iso']} | {e['id']} | {e['title'][:80]}")
            print(f"  latest  : {l['end_iso']} | {l['id']} | {l['title'][:80]}")
        else:
            print("  (no finished proposals)")


In [19]:
# --- Snapshot 투표 통계 수집기 (Jupyter 전용: argparse 없음) ---
# 기능(기존 동일) + ✅ discussion 있는 종료 제안만 처리

import os
import time
import random
from datetime import datetime, timezone
from typing import List, Dict, Any, Set, Tuple

import requests
import pandas as pd

# ===================== 사용자 설정 =====================
SNAPSHOT_API = "https://hub.snapshot.org/graphql"

SPACES: List[str] = [
    "aavedao.eth",
    "arbitrumfoundation.eth",
    "snapshot.dcl.eth",
    "balancer.eth",
    "cvx.eth",
    "1inch.eth",
    "aurafinance.eth",
    "lido-snapshot.eth",
    "uniswapgovernance.eth",
    "metislayer2.eth",
]

OUT_CSV = "dao_finished_proposals_stats.csv"   # 저장 파일명(상대/절대 경로 가능)
SPACE_COOLDOWN = (2.0, 3.0)                    # space 간 쿨다운 (초) 범위
VERBOSE = True                                  # 로그 출력 여부
# ======================================================

# ---- Rate limit / retry settings ----
MAX_RETRIES   = 6              # 429/5xx 재시도 횟수
BASE_SLEEP    = 0.8            # 요청 간 최소 대기(초)
BACKOFF_BASE  = 1.6            # 지수 백오프 배수
JITTER_RANGE  = (0.15, 0.45)   # 지터(랜덤) 범위(초)

session = requests.Session()
session.headers.update({
    "User-Agent": "agentics-snapshot-fetcher/0.5-jupyter (+github.com/you/yourrepo)"
})

# ---- GraphQL ----
# ✅ discussion 필드를 쿼리에 포함 (필터링에 사용)
PROPOSALS_Q = """
query($space: String!, $first: Int!, $skip: Int!) {
  proposals(
    first: $first
    skip: $skip
    where: { space_in: [$space] }
    orderBy: "created"
    orderDirection: desc
  ) {
    id
    title
    state
    end
    discussion  # ✅ 추가
  }
}
"""

# votes 쿼리는 created_gt / created_gte 두 가지 템플릿
VOTES_Q_GT  = """
query($id: String!, $first: Int!, $createdAfter: Int) {
  votes(
    first: $first
    where: { proposal: $id, created_gt: $createdAfter }
    orderBy: "created"
    orderDirection: asc
  ) {
    vp
    created
  }
}
"""
VOTES_Q_GTE = """
query($id: String!, $first: Int!, $createdAfter: Int) {
  votes(
    first: $first
    where: { proposal: $id, created_gte: $createdAfter }
    orderBy: "created"
    orderDirection: asc
  ) {
    vp
    created
  }
}
"""

# ---- Utils ----
def ts_to_iso(ts: int) -> str:
    return datetime.fromtimestamp(int(ts), tz=timezone.utc).isoformat()

def safe_sleep(seconds: float) -> None:
    time.sleep(seconds)

def gql(query: str, variables: Dict[str, Any] = None) -> Dict[str, Any]:
    """GraphQL 호출: 429/5xx 지수 백오프 재시도 + 요청 간 간격 유지."""
    variables = variables or {}
    retries = 0
    while True:
        safe_sleep(BASE_SLEEP + random.uniform(*JITTER_RANGE))
        try:
            r = session.post(
                SNAPSHOT_API,
                json={"query": query, "variables": variables},
                timeout=30
            )
        except requests.exceptions.RequestException:
            if retries < MAX_RETRIES:
                delay = (BACKOFF_BASE ** retries) + random.uniform(*JITTER_RANGE)
                safe_sleep(delay)
                retries += 1
                continue
            raise

        if r.status_code == 200:
            payload = r.json()
            if "errors" in payload:
                raise RuntimeError(payload["errors"])
            return payload["data"]

        if r.status_code in (429, 502, 503, 504):
            if retries < MAX_RETRIES:
                ra = r.headers.get("Retry-After")
                if ra:
                    try:
                        delay = float(ra)
                    except ValueError:
                        delay = (BACKOFF_BASE ** retries)
                else:
                    delay = (BACKOFF_BASE ** retries)
                delay += random.uniform(*JITTER_RANGE)
                safe_sleep(delay)
                retries += 1
                continue

        r.raise_for_status()

def fetch_all_proposals(space: str, batch: int = 100) -> List[Dict[str, Any]]:
    results, skip = [], 0
    while True:
        d = gql(PROPOSALS_Q, {"space": space, "first": batch, "skip": skip})
        chunk = d["proposals"]
        if not chunk:
            break
        results.extend(chunk)
        if len(chunk) < batch:
            break
        skip += batch
    return results

def fetch_all_votes(proposal_id: str, batch: int = 1000) -> List[Dict[str, Any]]:
    """created-커서 기반 페이지네이션."""
    results: List[Dict[str, Any]] = []
    created_after = None
    tried_gte = False

    while True:
        variables = {
            "id": proposal_id,
            "first": batch,
            "createdAfter": int(created_after) if created_after is not None else None
        }
        try:
            d = gql(VOTES_Q_GT if not tried_gte else VOTES_Q_GTE, variables)
        except RuntimeError:
            if not tried_gte:
                tried_gte = True
                d = gql(VOTES_Q_GTE, variables)
            else:
                raise

        chunk = d["votes"]
        if not chunk:
            break
        results.extend(chunk)

        last_created = chunk[-1].get("created")
        if last_created is None:
            break
        created_after = last_created

        if len(chunk) < batch:
            break

    return results

# ---- Resume helpers ----
def load_processed_ids(csv_path: str) -> Set[str]:
    """기존 CSV가 있으면 proposal_id 집합 로드 -> 재실행 시 skip."""
    if not os.path.exists(csv_path):
        return set()
    try:
        df = pd.read_csv(csv_path, usecols=["proposal_id"])
        return set(df["proposal_id"].astype(str).tolist())
    except Exception:
        return set()

def append_row(csv_path: str, row: Dict[str, Any]) -> None:
    """헤더 자동 관리하며 한 행씩 안전하게 append."""
    file_exists = os.path.exists(csv_path) and os.path.getsize(csv_path) > 0
    df = pd.DataFrame([row])
    df.to_csv(csv_path, mode="a", header=not file_exists, index=False)

# ---- Main worker ----
def summarize_spaces(
    spaces: List[str],
    out_csv: str,
    space_cooldown: Tuple[float, float] = (2.0, 3.0),
    verbose: bool = True
) -> int:
    now_ts = int(datetime.now(timezone.utc).timestamp())
    processed_ids = load_processed_ids(out_csv)

    total_new = 0
    for idx, space in enumerate(spaces, 1):
        if verbose:
            print(f"\n[{idx}/{len(spaces)}] {space} → proposals 가져오는 중…")

        try:
            proposals = fetch_all_proposals(space)
        except KeyboardInterrupt:
            print("\n⛔️ 사용자 중단 감지. 진행 현황은 CSV에 누적 저장되어 있습니다.")
            return total_new
        except Exception as e:
            print(f"  ! proposals fetch 실패: {e}")
            continue

        # 1) 종료된 proposal만
        finished = [p for p in proposals if p["state"] == "closed" and int(p["end"]) <= now_ts]

        # 2) ✅ discussion 필드가 비어있지 않은 것만 남김
        finished_with_discussion = [p for p in finished if (p.get("discussion") or "").strip()]

        # 재실행 skip: 이미 CSV에 들어간 proposal_id는 건너뜀
        to_process = [p for p in finished_with_discussion if str(p["id"]) not in processed_ids]

        if verbose:
            print(f"  finished (closed)           : {len(finished)}")
            print(f"  with discussion (subset)    : {len(finished_with_discussion)}")
            print(f"  ▶ 처리 예정(미저장·with_disc): {len(to_process)} "
                  f"(이미 저장 {len(finished_with_discussion)-len(to_process)}건 + "
                  f"discussion 없음 {len(finished)-len(finished_with_discussion)}건 skip)")

        # space 간 쿨다운
        safe_sleep(random.uniform(*space_cooldown))

        for j, prop in enumerate(to_process, 1):
            pid = str(prop["id"])
            title = prop.get("title", "")
            if verbose:
                print(f"    ({j}/{len(to_process)}) votes for {pid} …")

            try:
                votes = fetch_all_votes(pid)
            except KeyboardInterrupt:
                print("\n⛔️ 사용자 중단 감지. 현재까지의 행은 CSV에 저장되었습니다.")
                return total_new
            except Exception as e:
                print(f"      ! votes fetch 실패(pid={pid}): {e}")
                continue

            # vp 수치만 추출
            vps = []
            for v in votes:
                vp = v.get("vp")
                if vp is None:
                    continue
                try:
                    vps.append(float(vp))
                except (TypeError, ValueError):
                    continue

            if len(vps) == 0:
                stats = dict(min=0, q25=0, med=0, q75=0, max=0, mean=0, std=0, sum=0)
                num_voters = 0
            else:
                s = pd.Series(vps)
                stats = dict(
                    min=s.min(),
                    q25=s.quantile(0.25),
                    med=s.median(),
                    q75=s.quantile(0.75),
                    max=s.max(),
                    mean=s.mean(),
                    std=s.std(ddof=1),
                    sum=s.sum(),   # ✅ sp_sum
                )
                num_voters = int(s.size)

            row = {
                "space": space,
                "proposal_id": pid,
                "title": title,
                "end_iso": ts_to_iso(prop["end"]),
                "num_voters": num_voters,
                "vp_min": stats["min"],
                "vp_25%": stats["q25"],
                "vp_median": stats["med"],
                "vp_75%": stats["q75"],
                "vp_max": stats["max"],
                "vp_mean": stats["mean"],
                "vp_std": stats["std"],
                "sp_sum": stats["sum"],   # ✅ CSV 출력
            }

            # ✅ 한 행씩 즉시 저장 → 중간에 꺼져도 누적본 남음
            try:
                append_row(out_csv, row)
                processed_ids.add(pid)
                total_new += 1
            except Exception as e:
                print(f"      ! CSV 저장 실패(pid={pid}): {e}")

        # space 사이 쿨다운
        safe_sleep(random.uniform(*space_cooldown))

    return total_new


# ===================== 실행 (주피터 셀에서 그대로) =====================
print(f"작업 디렉토리: {os.getcwd()}")
print(f"CSV 절대 경로: {os.path.abspath(OUT_CSV)}")
print("Resume 방식: 기존 CSV의 proposal_id를 읽어 이미 처리한 제안은 자동 skip\n")

try:
    added_rows = summarize_spaces(
        spaces=SPACES,
        out_csv=OUT_CSV,
        space_cooldown=SPACE_COOLDOWN,
        verbose=VERBOSE
    )
    print(f"\n✅ 저장 완료: {OUT_CSV}  (이번 실행에서 추가된 행 수={added_rows})")
    print(f"CSV 절대 경로: {os.path.abspath(OUT_CSV)}")
except KeyboardInterrupt:
    print("\n⛔️ 사용자 중단. 현재까지의 결과는 CSV에 저장되어 있습니다.")


작업 디렉토리: /Users/chunghyunhan/Projects/agentics
CSV 절대 경로: /Users/chunghyunhan/Projects/agentics/dao_finished_proposals_stats.csv
Resume 방식: 기존 CSV의 proposal_id를 읽어 이미 처리한 제안은 자동 skip


[1/10] aavedao.eth → proposals 가져오는 중…
  finished (closed)           : 879
  with discussion (subset)    : 729
  ▶ 처리 예정(미저장·with_disc): 729 (이미 저장 0건 + discussion 없음 150건 skip)
    (1/729) votes for 0xa2b9d0717a82a111acc27e514bed07caa9b8636c12dd68fb61ae4dc57503c3cd …
    (2/729) votes for 0x4a9b7b0e64d20a9a7903ef82d20c7d3bc8e1612907187c55b41af66d5e0ec162 …
    (3/729) votes for 0x0c5427caf17d21b321a3b62362d085e580446b136b0eccf7f4dc377856025486 …
    (4/729) votes for 0x5e13f3e63fd9a2d4d00ff9f7915644e48d4b8b35fe03b52a599b4bc1c95914d0 …
    (5/729) votes for 0xbece41f2549d7b908a07ef1e5032e500e9c887b8252915d3782b74df35659d22 …
    (6/729) votes for 0xeb3572580924976867073ad9c8012cb9e52093c76dafebd7d3aebf318f2576fb …
    (7/729) votes for 0xfdcdcb613b3aaa3c60176369c27107c2de62a0bfb477fa792745d6148ed0249d …
